In [ ]:
!pip install accelerate -U
!pip install accelerate
!pip install transformers --upgrade
!pip install transformers[torch]
!pip install evaluate

import torch
import numpy as np
import pandas as pd
import evaluate
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Running on CPU")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 646.1/731.7 MB 23.7 MB/s eta 0:00:04
ERROR: Operation cancelled by user


In [ ]:
from torch.utils.data import random_split
from datasets import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [ ]:

# Data sourced from https://www.kaggle.com/datasets/thedrcat/daigt-v4-train-dataset, make sure to upload 2 csv files to google colab files

daigt_essays=pd.read_csv('train_v4_drcat_01.csv')
machine_essays = pd.read_csv('daigt_magic_generations.csv')
essays = pd.concat([machine_essays, daigt_essays], ignore_index=True)
dataset = Dataset.from_pandas(essays)







In [ ]:
import numpy as np

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

learning_rate = 2e-4
batch_size = 16
epochs = 2
#learning_rates = np.logspace(-3, -6, 4)

In [ ]:
# After 2 epcohs:   {'test_loss': 0.15880203247070312, 'test_accuracy': 0.9544011074016774, 'test_runtime': 189.5459, 'test_samples_per_second': 64.792, 'test_steps_per_second': 8.104}

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def process_data(data):
    encoded_data = tokenizer(data['text'], padding="max_length", truncation=True)
    return encoded_data

tokenized_datasets = dataset.map(process_data, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('prompt_name')
tokenized_datasets = tokenized_datasets.remove_columns('source')
tokenized_datasets = tokenized_datasets.remove_columns('RDizzl3_seven')

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(tokenized_datasets, [0.7, 0.15, 0.15])

# Warning: An error about missing "accelerate" package can eject when using CPU.

training_args = TrainingArguments(
    output_dir="/content/distilbert",
    per_device_train_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    save_strategy="epoch",
    evaluation_strategy="epoch",
)

# Use ADAM optimizer, which is selected by default
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

predictions = trainer.predict(test_dataset)
print(predictions.metrics)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/81879 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification


tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def process_data_bert(data):
    encoded_data = tokenizer(data['text'], padding="max_length", truncation=True)
    return encoded_data

tokenized_datasets = dataset.map(process_data_bert, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('prompt_name')
tokenized_datasets = tokenized_datasets.remove_columns('source')
tokenized_datasets = tokenized_datasets.remove_columns('RDizzl3_seven')


train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(tokenized_datasets, [0.7, 0.15, 0.15])

learning_rate = 2e-5
batch_size = 16
epochs = 2

#learning_rates = np.logspace(-3, -6, 4)

# Warning: An error about missing "accelerate" package can eject when using CPU.

training_args = TrainingArguments(
    output_dir="/content/bert",
    per_device_train_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    save_strategy="epoch",
    evaluation_strategy="epoch",
)

# Use ADAM optimizer, which is selected by default
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

predictions = trainer.predict(test_dataset)
print(predictions.metrics)

Map:   0%|          | 0/81879 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.064800,0.082344,0.978505
2,0.022000,0.112853,0.979075


{'test_loss': 0.09746795147657394, 'test_accuracy': 0.9813533099910431, 'test_runtime': 129.9507, 'test_samples_per_second': 94.505, 'test_steps_per_second': 11.82}


In [ ]:
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification
from accelerate import Accelerator

# Train with accelerator as model is too large
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def process_data_deberta(data):
    encoded_data = tokenizer(data['text'], padding="max_length", truncation=True, max_length=512)
    return encoded_data

tokenized_datasets = dataset.map(process_data_deberta, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('prompt_name')
tokenized_datasets = tokenized_datasets.remove_columns('source')
tokenized_datasets = tokenized_datasets.remove_columns('RDizzl3_seven')


train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(tokenized_datasets, [0.7, 0.15, 0.15])

learning_rate = 2e-5
batch_size = 4
epochs = 2
#learning_rates = np.logspace(-3, -6, 4)

# Warning: An error about missing "accelerate" package can eject when using CPU.

training_args = TrainingArguments(
    output_dir="/content/deberta",
    per_device_train_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    save_strategy="epoch",
    evaluation_strategy="epoch",
)

# Use ADAM optimizer, which is selected by default
model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=2).to(device)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

predictions = trainer.predict(test_dataset)
print(predictions.metrics)

Map:   0%|          | 0/81879 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.069700,0.124169,0.975574
2,0.002700,0.066736,0.989497


{'test_loss': 0.06497102230787277, 'test_accuracy': 0.9900659555410797, 'test_runtime': 191.2699, 'test_samples_per_second': 64.208, 'test_steps_per_second': 8.031}
